# Discord Bigram LDA — **Notebook Version**

This notebook reproduces your bigram LDA pipeline while **matching preprocessing** with the settings found in your `term_level_streamlined.ipynb`:
- `lowercase=True`, `keep_only_alpha=True`, `min_token_len=2`, `use_lemma=True`, `remove_numbers=True`
- NLTK stopwords **+** your `extra_stopwords` **+** `stopwords-iso.json` (if present)
- System message filters (same patterns)

> Generated automatically.

In [ ]:
# --- Config (ported to match your existing notebook) ---
import os
import re
import json
import math
from collections import Counter
from typing import Iterable, List, Tuple, Callable, Optional, Set, Dict, Any, Union

# Optional deps: NLTK stopwords
try:
    from nltk.corpus import stopwords as nltk_stop
    _NLTK_AVAILABLE = True
except Exception:
    _NLTK_AVAILABLE = False

# Optional deps: spaCy lemmatization
try:
    import spacy
    _SPACY_AVAILABLE = True
    try:
        nlp = spacy.load("en_core_web_sm")
    except Exception:
        # model not available; fall back to simple token logic
        _SPACY_AVAILABLE = False
        nlp = None
except Exception:
    _SPACY_AVAILABLE = False
    nlp = None


class Cfg: 
    # Flags
    lowercase: bool = True
    keep_only_alpha: bool = True
    min_token_len: int = 2
    use_lemma: bool = True
    remove_numbers: bool = True

    # Stopwords
    use_nltk_stopwords: bool = True
    extra_stopwords: Set[str] = {
        "server", "joined", "scroll", "papyrus", "image", "brett", "olsen",
        "entry", "start", "thread", "moshe", "levy", "casey", "handmer", "mae",
        "sawatzky", "like", "value", "seldon", "ben"
    }

    # stopwords-iso (optional)
    stopwords_iso: Dict[str, List[str]] = {}
    try:
        with open("stopwords-iso.json", "r", encoding="utf-8") as _f:
            stopwords_iso = json.load(_f)
    except Exception:
        stopwords_iso = {}

    # Filters / regexes
    system_message_patterns = [
        re.compile(r"^\s*Joined the server\.?\s*$", re.I),
        re.compile(r"^\s*Started a thread\.?\s*$", re.I),
    ]
    url_inline_re = re.compile(r"(https?://\S+|www\.\S+)", re.I)
    url_only_re = re.compile(r"^\s*(https?://\S+\s*)+$", re.I)
    emoji_re = re.compile(
        "["                      # emojis
        "\U0001F300-\U0001F5FF"
        "\U0001F600-\U0001F64F"
        "\U0001F680-\U0001F6FF"
        "\U0001F700-\U0001F77F"
        "\U0001F780-\U0001F7FF"
        "\U0001F800-\U0001F8FF"
        "\U0001F900-\U0001F9FF"
        "\U0001FA00-\U0001FA6F"
        "\U0001FA70-\U0001FAFF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+"
    )
    greek_re = re.compile(r"[\u0370-\u03FF\u1F00-\u1FFF]")


cfg = Cfg()


def contains_greek(text: str) -> bool:
    return bool(cfg.greek_re.search(text or ""))


# --- Stopwords assembly (NLTK + extra + stopwords-iso) ---
def build_stopword_set() -> Set[str]:
    sw: Set[str] = set()

    # Include NLTK stopwords
    if cfg.use_nltk_stopwords and _NLTK_AVAILABLE:
        try:
            sw |= set(nltk_stop.words("english"))
        except Exception:
            pass

    # Include extra custom stopwords
    sw |= set(cfg.extra_stopwords)

    # Include stopwords from stopwords-iso.json (English + fallback for others)
    if isinstance(cfg.stopwords_iso, dict):
        if "en" in cfg.stopwords_iso:
            sw |= set(cfg.stopwords_iso["en"])
        else:
            # fallback: flatten all lists if language keys differ
            for lang_words in cfg.stopwords_iso.values():
                try:
                    sw |= set(lang_words)
                except Exception:
                    pass

    # Normalize everything to lowercase
    return {w.lower() for w in sw}


STOPWORDS = build_stopword_set()
# print(len(STOPWORDS))  # uncomment for sanity count


# --- System / URL / emoji cleaning ---
def is_system_message(text: str) -> bool:
    t = text or ""
    for pat in cfg.system_message_patterns:
        if pat.search(t):
            return True
    return False


def strip_urls(text: str) -> str:
    t = text or ""
    if cfg.url_only_re.match(t):
        return ""
    return cfg.url_inline_re.sub(" ", t)


def strip_emoji(text: str) -> str:
    return cfg.emoji_re.sub(" ", text or "")


def clean_text(text: str) -> str:
    if not text:
        return ""
    if is_system_message(text):
        return ""
    t = strip_urls(text)
    t = strip_emoji(t)
    t = t.replace("’", "'")  # normalize apostrophe
    return re.sub(r"\s+", " ", t).strip()


# --- Tokenization / normalization (spaCy if available) ---
def doc_to_tokens(doc_text: str, nlp_obj: Any, stopwords: Set[str]) -> List[str]:
    t = clean_text(doc_text)
    if not t:
        return []
    t = t.lower()  # lowercase before spaCy to stabilize POS/lemmatization

    toks: List[str] = []
    if _SPACY_AVAILABLE and nlp_obj is not None:
        sp = nlp_obj(t)
        for token in sp:
            txt = token.text
            if cfg.keep_only_alpha and not txt.isalpha():
                continue
            if cfg.remove_numbers and any(ch.isdigit() for ch in txt):
                continue

            raw = txt.lower() if cfg.lowercase else txt
            norm = token.lemma_.lower() if (cfg.use_lemma and token.lemma_) else raw

            if contains_greek(norm):
                norm = "GREEK_LETTERS"  # keep uppercase on purpose

            if len(norm) < cfg.min_token_len:
                continue
            if norm in stopwords:
                continue
            toks.append(norm)
    else:
        # Simple fallback (no lemma)
        words = re.findall(r"(?u)\b[\w']+\b", t)
        for w in words:
            if cfg.keep_only_alpha and not w.isalpha():
                continue
            if cfg.remove_numbers and any(ch.isdigit() for ch in w):
                continue
            tok = w.lower() if cfg.lowercase else w
            if contains_greek(tok):
                tok = "GREEK_LETTERS"
            if len(tok) < cfg.min_token_len:
                continue
            if tok in stopwords:
                continue
            toks.append(tok)

    return toks


def docs_to_token_lists(texts: Iterable[str]) -> List[List[str]]:
    return [doc_to_tokens(t, nlp, STOPWORDS) for t in texts]


def bigrams_from_tokens_per_doc(tokens_per_doc: List[List[str]]) -> List[List[Tuple[str, str]]]:
    out: List[List[Tuple[str, str]]] = []
    for toks in tokens_per_doc:
        out.append([(toks[i], toks[i + 1]) for i in range(len(toks) - 1)] if len(toks) > 1 else [])
    return out


def unigram_counts(tokens_per_doc: List[List[str]]) -> Tuple[Counter, Counter]:
    tf, df = Counter(), Counter()
    for toks in tokens_per_doc:
        tf.update(toks)
        df.update(set(toks))
    return tf, df


def bigram_counts(bigrams_per_doc: List[List[Tuple[str, str]]]) -> Tuple[Counter, Counter]:
    tf, df = Counter(), Counter()
    for bigs in bigrams_per_doc:
        tf.update(bigs)
        df.update(set(bigs))
    return tf, df


def mutual_information_bigrams(unigram_tf: Counter, bigram_tf: Counter, smoothing: int = 1) -> Dict[Tuple[str, str], float]:
    T = sum(bigram_tf.values()) + smoothing
    mi: Dict[Tuple[str, str], float] = {}
    for (w1, w2), c12 in bigram_tf.items():
        c1 = unigram_tf.get(w1, 0)
        c2 = unigram_tf.get(w2, 0)
        num = (c12 + smoothing) * T
        den = (c1 + smoothing) * (c2 + smoothing)
        mi[(w1, w2)] = math.log2(float(num) / float(den))
    return mi


# --- Corpus analysis (unigrams, bigrams, MI) ---
def analyze_corpus(
    docs: Union[Iterable[str], List[List[str]]],
    top_k: int = 5000,
    output_dir: Optional[str] = None,
    pretokenized: bool = False
) -> Dict[str, Any]:

    import pandas as pd

    if pretokenized:
        tokens_per_doc = docs  # type: ignore[assignment]
    else:
        tokens_per_doc = docs_to_token_lists(docs)  # type: ignore[arg-type]

    bigrams_per_doc = bigrams_from_tokens_per_doc(tokens_per_doc)
    uni_tf, uni_df = unigram_counts(tokens_per_doc)
    bi_tf, bi_df = bigram_counts(bigrams_per_doc)

    df_uni = (
        pd.DataFrame([(t, uni_tf[t], uni_df[t]) for t in uni_tf], columns=["term", "tf", "df"])
        .sort_values(["tf", "df", "term"], ascending=[False, False, True])
        .head(top_k)
    )
    df_bi = (
        pd.DataFrame([(" ".join(p), bi_tf[p], bi_df[p]) for p in bi_tf], columns=["bigram", "tf", "df"])
        .sort_values(["tf", "df", "bigram"], ascending=[False, False, True])
        .head(top_k)
    )

    mi = mutual_information_bigrams(uni_tf, bi_tf, smoothing=1)
    df_mi = (
        pd.DataFrame([(f"{w1} {w2}", bi_tf[(w1, w2)], m) for (w1, w2), m in mi.items()],
                     columns=["bigram", "tf", "mi"])
        .sort_values(["mi", "tf", "bigram"], ascending=[False, False, True])
        .head(top_k)
    )

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        df_uni.to_csv(os.path.join(output_dir, "unigrams.csv"), index=False)
        df_bi.to_csv(os.path.join(output_dir, "bigrams.csv"), index=False)
        df_mi.to_csv(os.path.join(output_dir, "bigrams_mi.csv"), index=False)

    return {"unigrams": df_uni, "bigrams": df_bi, "bigrams_mi": df_mi}


# --- JSON loading with tokens and cleaned context ---
def load_json_records_with_tokens(folder: str) -> List[Dict[str, Any]]:

    records: List[Dict[str, Any]] = []
    try:
        files = [f for f in os.listdir(folder) if f.lower().endswith(".json")]
    except Exception as e:
        print(f"⚠️ Could not list folder '{folder}': {e}")
        return records

    for fname in files:
        fpath = os.path.join(folder, fname)
        try:
            with open(fpath, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print(f"⚠️ Skipped {fname}: {e}")
            continue

        channel = (data.get("channel", {}) or {}).get("name") or data.get("name") or os.path.splitext(fname)[0]
        for m in (data.get("messages", []) or []):
            raw = (m.get("content") or "").strip()
            if not raw:
                continue

            tokens = doc_to_tokens(raw, nlp, STOPWORDS)
            if not tokens:
                continue

            author = (m.get("author") or {}).get("name") or (m.get("author") or {}).get("id") or "Unknown"
            ts = m.get("timestamp") or ""
            records.append({
                "author": author,
                "timestamp": ts,
                "channel": channel,
                "raw": raw,
                "tokens": tokens,
                "context": ", ".join(tokens),  # comma-separated tokens
            })

    print(f"Loaded {len(records)} normalized messages from {len(files)} files.")
    return records

# --- Example usage (commented) ---
# records = load_json_records_with_tokens("/path/to/filtered_JSON")
# docs_tokens = [r["tokens"] for r in records]
# results = analyze_corpus(docs_tokens, top_k=100000, output_dir="/path/to/outputs", pretokenized=True)
# import pandas as pd
# df_msgs = pd.DataFrame([{k: r[k] for k in ["author","timestamp","channel","raw","context"]} for r in records],
#                        columns=["author","timestamp","channel","raw","context"])
# df_msgs.head()


/Users/nikhil/PycharmProjects/vesuvius_discord_study/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# --- System message filters ---
system_message_patterns = [
        re.compile(r"^\s*Joined the server\.?\s*$", re.I),
        re.compile(r"^\s*Started a thread\.?\s*$", re.I),
    ]

def is_system_message(text: str) -> bool:
    for pat in system_message_patterns:
        if pat.match(text or ""):
            return True
    return False

In [3]:
# --- Load Discord JSON and clean ---
import pandas as pd
import numpy as np

URL_ONLY_RE = re.compile(r"^\s*(?:https?://\S+|www\.\S+)(?:\s+(?:https?://\S+|www\.\S+))*\s*$", re.I)
URL_INLINE_RE = re.compile(r"(https?://\S+|www\.\S+)", re.I)

def load_discord_dir(input_dir: str) -> pd.DataFrame:
    rows = []
    p = os.path.expanduser(input_dir)
    for root, _, files in os.walk(p):
        for fname in files:
            if not fname.lower().endswith(".json"):
                continue
            fpath = os.path.join(root, fname)
            try:
                with open(fpath, "r", encoding="utf-8") as f:
                    data = json.load(f)
            except Exception:
                continue
            guild = data.get("guild", {}) or {}
            channel = data.get("channel", {}) or {}
            for m in (data.get("messages", []) or []):
                txt = m.get("content")
                rows.append({
                    "guild_id": guild.get("id"),
                    "guild_name": guild.get("name"),
                    "channel_id": channel.get("id"),
                    "channel_name": channel.get("name"),
                    "category": channel.get("category"),
                    "message_id": m.get("id"),
                    "timestamp": m.get("timestamp"),
                    "text": "" if txt is None else str(txt),
                })
    df = pd.DataFrame(rows)
    if df.empty:
        return df
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)
    df["text"] = df["text"].astype(str)
    return df

def clean_messages(df: pd.DataFrame, min_doc_len: int = 10) -> pd.DataFrame:
    if df.empty:
        return df
    df = df.copy()
    # base trimming
    df["text"] = df["text"].fillna("").astype(str).str.strip()
    df = df[df["text"] != ""]
    # system boilerplate
    df = df[~df["text"].apply(is_system_message)]
    # url-only rows
    df = df[~df["text"].str.fullmatch(URL_ONLY_RE, na=False)]
    # strip inline URLs
    df["text"] = df["text"].str.replace(URL_INLINE_RE, "", regex=True).str.strip()
    # normalize curly apostrophes
    df["text"] = df["text"].str.replace("’", "'", regex=False)
    # enforce min length
    df = df[df["text"].str.len() >= int(min_doc_len)]
    return df

In [4]:
# --- Tokenization & analyzer (matches flags from your notebook) ---
WORD_RE = re.compile(r"(?u)\b[\w']+\b")

def _lemma_tokens_spacy(text: str):
    if not _SPACY_AVAILABLE or nlp is None:
        return None
    doc = nlp(text)
    out = []
    for t in doc:
        w = t.lemma_ if getattr(cfg, "use_lemma", True) else t.text
        out.append(w)
    return out

def base_tokens(text: str):
    t = text or ""
    if getattr(cfg, "lowercase", True):
        t = t.lower()
    # optional: remove digits before tokenizing
    if getattr(cfg, "remove_numbers", True):
        t = re.sub(r"\d+", " ", t)
    # spaCy lemma fallback
    toks = _lemma_tokens_spacy(t)
    if toks is None:
        toks = WORD_RE.findall(t)
    # keep-only-alpha filter
    if getattr(cfg, "keep_only_alpha", True):
        toks = [w for w in toks if re.fullmatch(r"[a-zA-Z']+", w or "")]
    # min token length
    mlen = int(getattr(cfg, "min_token_len", 2))
    toks = [w for w in toks if isinstance(w, str) and len(w) >= mlen]
    # stopwords
    toks = [w for w in toks if w not in STOPWORDS]
    return toks

def bigram_analyzer(doc: str):
    toks = base_tokens(doc)
    return [f"{a} {b}" for a, b in zip(toks, toks[1:])]

In [5]:
# --- Time aggregation (per-channel buckets) ---
def aggregate_messages(df: pd.DataFrame, window: str) -> pd.DataFrame:
    dfa = df.copy()
    dfa["bucket_start"] = pd.to_datetime(dfa["timestamp"], errors="coerce", utc=True).dt.floor(window)
    keys = ["channel_id", "bucket_start"]
    agg = (
        dfa.sort_values("timestamp")
           .groupby(keys, as_index=False)
           .agg(
               text=("text", lambda s: " ".join([t for t in s if isinstance(t, str) and t.strip()])),
               channel_name=("channel_name", "first"),
               category=("category", "first"),
               n_msgs=("text", "count")
           )
    )
    agg = agg[agg["text"].str.len() >= 10]
    return agg

def label_corpora(df_docs: pd.DataFrame, pap_pat: str, gen_pat: str) -> pd.Series:
    papy = df_docs["channel_name"].fillna("").str.contains(pap_pat, case=False, regex=True) | \
           df_docs["category"].fillna("").str.contains(pap_pat, case=False, regex=True)
    gen  = df_docs["channel_name"].fillna("").str.contains(gen_pat, case=False, regex=True) | \
           df_docs["category"].fillna("").str.contains(gen_pat, case=False, regex=True)
    labels = pd.Series(np.nan, index=df_docs.index, dtype="float")
    labels[papy] = 1.0
    labels[gen & ~papy] = 0.0
    return labels

In [6]:
# --- Vectorization, LDA, and exports ---
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def build_bigram_matrix(docs, min_df=2, max_df=0.90, use_tfidf=False):
    """Build bigram matrix using either raw counts or TF-IDF weighting.
    
    Args:
        docs: List of document strings
        min_df: Minimum document frequency threshold
        max_df: Maximum document frequency threshold
        use_tfidf: If True, use TF-IDF weighting instead of raw counts
    """
    if use_tfidf:
        from sklearn.feature_extraction.text import TfidfVectorizer
        vect = TfidfVectorizer(analyzer=bigram_analyzer, lowercase=False, min_df=min_df, max_df=max_df)
    else:
        vect = CountVectorizer(analyzer=bigram_analyzer, lowercase=False, min_df=min_df, max_df=max_df)
    X = vect.fit_transform(docs)
    return vect, X

def top_terms_from_components(components, feature_names, topn=15):
    topic_terms = []
    for comp in components:
        idx = np.argsort(comp)[::-1][:topn]
        terms = [(feature_names[i], float(comp[i])) for i in idx]
        topic_terms.append(terms)
    return topic_terms

def umass_coherence(components, X, topn=10, eps=1.0):
    Xb = (X > 0).astype(int)
    df = np.asarray(Xb.sum(axis=0)).ravel()
    Xb_csr = Xb.tocsr()
    coherences = []
    for comp in components:
        ids = np.argsort(comp)[::-1][:topn]
        score = 0.0
        pairs = 0
        for i in range(len(ids)):
            for j in range(i+1, len(ids)):
                ti, tj = ids[i], ids[j]
                docs_i = set(Xb_csr[:, ti].nonzero()[0])
                docs_j = set(Xb_csr[:, tj].nonzero()[0])
                dij = len(docs_i & docs_j)
                score += np.log((dij + eps) / (df[ti] + eps))
                pairs += 1
        coherences.append(float(score / max(pairs, 1)))
    return coherences

def summarize_topic_sizes(theta):
    hard = theta.argmax(axis=1)
    K = theta.shape[1]
    hard_count = np.bincount(hard, minlength=K)
    mean_weight = theta.mean(axis=0)
    return {"hard_count": hard_count, "mean_weight": mean_weight}

def per_corpus_distribution(theta, labels):
    hard = theta.argmax(axis=1)
    K = theta.shape[1]
    rows = []
    for k in range(K):
        mask_k = (hard == k)
        papy = int(np.sum(mask_k & (labels == 1)))
        gen  = int(np.sum(mask_k & (labels == 0)))
        rows.append({"topic": k, "docs_papyrology": papy, "docs_general": gen})
    return pd.DataFrame(rows)

def run_pipeline(
    input_dir: str,
    output_dir: str = "./outputs_rq2",
    aggregate_window: str = "",
    ks = (20,25,30),
    max_df: float = 0.90,
    min_df: int = 2,
    topn: int = 15,
    papyrology_pattern: str = r"papyrolog",
    general_pattern: str = r"general",
    min_doc_len: int = 10,
    use_tfidf: bool = False
):
    outdir = os.path.expanduser(output_dir)
    os.makedirs(outdir, exist_ok=True)
    print("[1/5] Load & clean…")
    df = load_discord_dir(input_dir)
    if df.empty:
        print("No messages found.")
        return None
    df = clean_messages(df, min_doc_len=min_doc_len)
    if df.empty:
        print("All messages filtered out.")
        return None

    if aggregate_window:
        print(f"[2/5] Aggregating by {aggregate_window} per channel…")
        df_docs = aggregate_messages(df, aggregate_window)
        docs = df_docs["text"].tolist()
        timestamps = df_docs["bucket_start"].tolist()
    else:
        print("[2/5] No aggregation (per-message documents).")
        df_docs = df.copy()
        docs = df_docs["text"].tolist()
        timestamps = df_docs["timestamp"].tolist()

    print(f"Documents for LDA: {len(docs):,}")

    labels = label_corpora(df_docs, papyrology_pattern, general_pattern).to_numpy()

    print(f"[3/5] Building bigram matrix… (matching your tokenizer/stopwords)")
    if use_tfidf:
        print("  Using TF-IDF weighting (frequent terms will be downweighted)")
    else:
        print("  Using raw counts")
    vect, X = build_bigram_matrix(docs, min_df=min_df, max_df=max_df, use_tfidf=use_tfidf)
    feature_names = vect.get_feature_names_out()
    print(f"Vocabulary size (bigrams): {len(feature_names):,}")
    
    # For coherence calculation, we need binary (presence/absence) matrix
    # Convert to binary if using TF-IDF
    X_binary = (X > 0).astype(int) if use_tfidf else X

    outputs = []
    for k in ks:
        print(f"[4/5] Fitting LDA with k={k}…")
        lda = LatentDirichletAllocation(
            n_components=k,
            learning_method="batch",
            random_state=42,
            doc_topic_prior=None,
            topic_word_prior=None,
            max_iter=50,
            evaluate_every=0,
        )
        lda.fit(X)
        theta = lda.transform(X)
        comps = lda.components_

        topic_terms = top_terms_from_components(comps, feature_names, topn=topn)
        topic_sizes = summarize_topic_sizes(theta)
        hard_count = topic_sizes["hard_count"]
        mean_weight = topic_sizes["mean_weight"]

        print("  Computing UMass coherence…")
        # Use binary matrix for coherence (presence/absence)
        coh = umass_coherence(comps, X_binary, topn=min(topn, 10))

        comp_df = per_corpus_distribution(theta, labels)

        rows = []
        for t_id, terms in enumerate(topic_terms):
            top_terms = ", ".join([w for (w, _) in terms])
            top_probs = ", ".join([f"{w}:{wt:.3f}" for (w, wt) in terms])
            pap = int(comp_df.loc[comp_df["topic"] == t_id, "docs_papyrology"].iloc[0]) if not comp_df.empty else 0
            gen = int(comp_df.loc[comp_df["topic"] == t_id, "docs_general"].iloc[0]) if not comp_df.empty else 0
            rows.append({
                "k": k,
                "topic": t_id,
                "docs_hard_count": int(hard_count[t_id]),
                "docs_mean_weight": float(mean_weight[t_id]),
                "coherence_umass": float(coh[t_id]),
                "top_terms": top_terms,
                "top_terms_with_weights": top_probs,
                "docs_papyrology": pap,
                "docs_general": gen
            })
        topics_df = pd.DataFrame(rows).sort_values("docs_hard_count", ascending=False)
        out_csv = os.path.join(outdir, f"topics_k{k}.csv")
        topics_df.to_csv(out_csv, index=False)
        print("  Saved:", out_csv)
        outputs.append(out_csv)

    print("\n[5/5] Done ✓")
    print(f"- Docs: {len(docs):,}")
    print(f"- Vocab(bigrams): {len(feature_names):,}")
    print(f"- Ks tried: {list(ks)}")
    print(f"- Outputs in: {os.path.abspath(outdir)}")
    print(f"- Aggregation: {'ON ('+aggregate_window+')' if aggregate_window else 'OFF'}")
    print(f"- Weighting: {'TF-IDF' if use_tfidf else 'Raw counts'}")
    print(f"- Lemmatization: {'ON (spaCy)' if _SPACY_AVAILABLE else 'OFF (fallback tokens)'}")
    print(f"- lower={getattr(cfg, 'lowercase', True)}, keep_only_alpha={getattr(cfg, 'keep_only_alpha', True)}, min_token_len={getattr(cfg, 'min_token_len', 2)}, remove_numbers={getattr(cfg, 'remove_numbers', True)}")
    return outputs

In [7]:
# --- Example: run the pipeline (edit paths as needed) ---
outputs = run_pipeline(
    input_dir="./filtered_JSON",
    output_dir="./outputs_rq2",
    aggregate_window="30min",
    ks=(20,25,30),
    max_df=0.90,
    min_df=2,
    topn=15,
    papyrology_pattern=r"papyrolog",
    general_pattern=r"general",
    min_doc_len=10,
)
outputs

[1/5] Load & clean…
[2/5] Aggregating by 30min per channel…
Documents for LDA: 1,976
[3/5] Building bigram matrix… (matching your tokenizer/stopwords)
  Using raw counts
Vocabulary size (bigrams): 3,013
[4/5] Fitting LDA with k=20…
  Computing UMass coherence…
  Saved: ./outputs_rq2/topics_k20.csv
[4/5] Fitting LDA with k=25…
  Computing UMass coherence…
  Saved: ./outputs_rq2/topics_k25.csv
[4/5] Fitting LDA with k=30…
  Computing UMass coherence…
  Saved: ./outputs_rq2/topics_k30.csv

[5/5] Done ✓
- Docs: 1,976
- Vocab(bigrams): 3,013
- Ks tried: [20, 25, 30]
- Outputs in: /Users/nikhil/PycharmProjects/vesuvius_discord_study/term_level/outputs_rq2
- Aggregation: ON (30min)
- Weighting: Raw counts
- Lemmatization: ON (spaCy)
- lower=True, keep_only_alpha=True, min_token_len=2, remove_numbers=True


['./outputs_rq2/topics_k20.csv',
 './outputs_rq2/topics_k25.csv',
 './outputs_rq2/topics_k30.csv']

# 🔍 Issue Analysis: Why "ink detection" appears in multiple topics

**Problem:** Very frequent terms (like "ink detection") can appear across many topics in LDA because:
1. They appear in many documents (high document frequency)
2. LDA probabilistically assigns common terms to multiple topics to explain the data
3. The current `max_df=0.90` only filters terms appearing in >90% of documents

**Solutions:**
1. **Lower `max_df`** to filter out more common terms (e.g., 0.50-0.60)
2. **Use TF-IDF** instead of raw counts to downweight frequent terms
3. **Add to stopwords** if it's domain-specific noise

Let's first check the actual document frequency of "ink detection":


In [8]:
# Diagnostic: Check document frequency of "ink detection"
from sklearn.feature_extraction.text import CountVectorizer

# Re-run the pipeline setup to get the vectorizer
input_dir = "./filtered_JSON"
df = load_discord_dir(input_dir)
df = clean_messages(df, min_doc_len=10)
df_docs = aggregate_messages(df, "30min")
docs = df_docs["text"].tolist()

# Build vectorizer with current settings
vect, X = build_bigram_matrix(docs, min_df=2, max_df=0.90)
feature_names = vect.get_feature_names_out()

# Find "ink detection" in vocabulary
ink_detection_idx = None
for i, name in enumerate(feature_names):
    if name == "ink detection":
        ink_detection_idx = i
        break

if ink_detection_idx is not None:
    # Calculate document frequency (how many documents contain this term)
    X_binary = (X[:, ink_detection_idx] > 0).astype(int)
    doc_freq = X_binary.sum()
    doc_freq_pct = (doc_freq / len(docs)) * 100
    
    # Calculate term frequency (total occurrences)
    term_freq = X[:, ink_detection_idx].sum()
    
    print(f"📊 'ink detection' Statistics:")
    print(f"  - Document Frequency: {doc_freq:,} / {len(docs):,} documents ({doc_freq_pct:.1f}%)")
    print(f"  - Term Frequency: {term_freq:,} total occurrences")
    print(f"  - Current max_df threshold: 0.90 ({int(0.90 * len(docs))} docs)")
    print(f"\n✅ 'ink detection' appears in {doc_freq_pct:.1f}% of documents")
    print(f"   With max_df=0.90, it's kept because {doc_freq_pct:.1f}% < 90%")
    print(f"\n💡 Solution: Lower max_df to {(doc_freq_pct/100):.2f} or below to filter it out")
else:
    print("⚠️ 'ink detection' not found in vocabulary (may have been filtered)")


📊 'ink detection' Statistics:
  - Document Frequency: 126 / 1,976 documents (6.4%)
  - Term Frequency: 150 total occurrences
  - Current max_df threshold: 0.90 (1778 docs)

✅ 'ink detection' appears in 6.4% of documents
   With max_df=0.90, it's kept because 6.4% < 90%

💡 Solution: Lower max_df to 0.06 or below to filter it out


## Solution 1: Lower `max_df` to filter common terms

Run the pipeline with a lower `max_df` (e.g., 0.50 or 0.60) to filter out terms that appear in more than 50-60% of documents. This will remove "ink detection" and other overly common terms.


In [9]:
# #Solution 1: Re-run with lower max_df (uncomment to run)
# outputs_filtered = run_pipeline(
#     input_dir="./filtered_JSON",
#     output_dir="./outputs_rq2_filtered",  # Different output directory
#     aggregate_window="30min",
#     ks=(20,25,30),
#     max_df=0.50,  # Lower threshold - filters terms in >50% of documents
#     min_df=2,
#     topn=15,
#     papyrology_pattern=r"papyrolog",
#     general_pattern=r"general",
#     min_doc_len=10,
# )
# outputs_filtered


## Solution 2: Use TF-IDF weighting instead of raw counts

TF-IDF (Term Frequency-Inverse Document Frequency) automatically downweights frequent terms, which naturally reduces their dominance across topics. This requires modifying the LDA pipeline to use `TfidfVectorizer` instead of `CountVectorizer`.


In [10]:
# Solution 2: TF-IDF version - Now integrated into run_pipeline()!
# The run_pipeline() function now accepts use_tfidf=True parameter
# TF-IDF automatically downweights frequent terms like "ink detection"


## Solution 3: Add "ink detection" to stopwords

If "ink detection" is domain-specific noise that doesn't help distinguish topics, you can add it directly to the stopword list in the configuration.


In [11]:
# Solution 3: Add "ink detection" to stopwords
# In the Cfg class above, modify extra_stopwords:
# extra_stopwords: Set[str] = {
#     "server", "joined", "scroll", "papyrus", "image", "brett", "olsen",
#     "entry", "start", "thread", "moshe", "levy", "casey", "handmer", "mae",
#     "sawatzky", "like", "value", "seldon", "ben",
#     "ink", "detection"  # Add individual words, or filter bigrams separately
# }

# However, since you're using bigrams, you'd need to filter "ink detection" 
# after bigram generation. A better approach is Solution 1 (lower max_df).


## ✅ Run LDA with TF-IDF

Use `use_tfidf=True` in `run_pipeline()` to apply TF-IDF weighting. This will automatically downweight frequent terms like "ink detection", reducing their dominance across topics.


In [12]:
# Run LDA with TF-IDF weighting
outputs_tfidf = run_pipeline(
    input_dir="./filtered_JSON",
    output_dir="./outputs_rq2_tfidf",  # Different output directory
    aggregate_window="30min",
    ks=(20,25,30),
    max_df=0.90,  # Can keep max_df=0.90 since TF-IDF will downweight frequent terms
    min_df=2,
    topn=15,
    papyrology_pattern=r"papyrolog",
    general_pattern=r"general",
    min_doc_len=10,
    use_tfidf=True  # 🎯 Enable TF-IDF weighting
)
outputs_tfidf


[1/5] Load & clean…
[2/5] Aggregating by 30min per channel…
Documents for LDA: 1,976
[3/5] Building bigram matrix… (matching your tokenizer/stopwords)
  Using TF-IDF weighting (frequent terms will be downweighted)
Vocabulary size (bigrams): 3,013
[4/5] Fitting LDA with k=20…
  Computing UMass coherence…
  Saved: ./outputs_rq2_tfidf/topics_k20.csv
[4/5] Fitting LDA with k=25…
  Computing UMass coherence…
  Saved: ./outputs_rq2_tfidf/topics_k25.csv
[4/5] Fitting LDA with k=30…
  Computing UMass coherence…
  Saved: ./outputs_rq2_tfidf/topics_k30.csv

[5/5] Done ✓
- Docs: 1,976
- Vocab(bigrams): 3,013
- Ks tried: [20, 25, 30]
- Outputs in: /Users/nikhil/PycharmProjects/vesuvius_discord_study/term_level/outputs_rq2_tfidf
- Aggregation: ON (30min)
- Weighting: TF-IDF
- Lemmatization: ON (spaCy)
- lower=True, keep_only_alpha=True, min_token_len=2, remove_numbers=True


['./outputs_rq2_tfidf/topics_k20.csv',
 './outputs_rq2_tfidf/topics_k25.csv',
 './outputs_rq2_tfidf/topics_k30.csv']